In [1]:
import pandas as pd
import numpy as np

CALL = pd.read_csv('lv2-2502.csv')
print(CALL.shape)
print(CALL.info())

(270, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 7 columns):
CS_DT       270 non-null object
AID         270 non-null int64
PID         270 non-null int64
CALL_CNT    270 non-null int64
TM_AVG      270 non-null float64
PASS        270 non-null int64
DAT_ROLE    270 non-null object
dtypes: float64(1), int64(4), object(2)
memory usage: 14.9+ KB
None


### Question 24

In [2]:
# 24-1
CALL_AGG = CALL.groupby('PID').agg({'TM_AVG':('mean','std')}).reset_index()
CALL_AGG.columns = ['PID','MEAN','STD']
CALL_AGG['CV'] = CALL_AGG['STD']/CALL_AGG['MEAN']
CALL_AGG

,PID,MEAN,STD,CV
0,1,1236.670333,530.635472,0.429084
1,2,491.699778,381.040459,0.774945
2,3,345.131889,298.498611,0.864883


In [3]:
# 24-2
A4 = np.min(abs(CALL_AGG['CV']))
B4 = CALL_AGG.loc[abs(CALL_AGG['CV']).idxmin()].PID
C4 = A4.astype('float') + B4
print('A4:',A4)
print('B4:',B4)
print('C4:',C4)

A4: 0.429084015436806
B4: 1.0
C4: 1.429084015436806


In [4]:
# Answer 24
ans24f = C4
ans24 = round(ans24f, 3)

print(ans24f)
print(ans24)

1.429084015436806
1.429


### Question 25

In [5]:
# 25-1
DF250 = CALL.copy()
DF250['D_P_LEN'] = DF250['CALL_CNT'] * DF250['TM_AVG']

In [6]:
# 25-2
DF25 = DF250.groupby('AID').agg({'CALL_CNT':'sum','D_P_LEN':'sum','PASS':'mean'}).reset_index()
DF25.columns = ['AID','A_CNT','A_LEN','A_PASS']


In [7]:
# 25-3
DF25['A_AVG'] = DF25['A_LEN']/DF25['A_CNT']
DF25

,AID,A_CNT,A_LEN,A_PASS,A_AVG
0,1,188,254977.86,0.000000,1356.265213
1,2,116,27502.85,0.000000,237.093534
2,3,401,367339.91,0.666667,916.059626
3,4,311,105158.02,0.629630,338.128682
4,5,580,441875.93,0.666667,761.855052
5,6,496,220057.96,1.000000,443.665242
6,7,759,540201.09,0.740741,711.727391
7,8,695,366476.26,0.851852,527.303971
8,9,966,733237.33,0.666667,759.044855
9,10,886,504662.32,0.851852,569.596298


In [8]:
# 25-3
from sklearn.preprocessing import StandardScaler
svar = ['A_CNT','A_LEN']
svars = [f"{x}_S" for x in svar]
scaler = StandardScaler().fit(DF25[svar])
DF25s = pd.DataFrame(scaler.transform(DF25[svar]), columns = svars)

DF25f = pd.concat([DF25, DF25s], axis = 1)

In [9]:
# 25-4
from sklearn.cluster import KMeans 
KM254 = KMeans(random_state=1234, n_clusters=2).fit(DF25f[svars])
DF25f['CLST'] = KM254.labels_

In [10]:
# 25-5
DF25_AGG = DF25f.groupby('CLST').A_PASS.mean()
print(DF25_AGG)
abs(max(DF25_AGG) - min(DF25_AGG))

CLST
0    0.459259
1    0.755556
Name: A_PASS, dtype: float64


0.2962962962962963

In [11]:
# Answer 25
ans25f = abs(max(DF25_AGG) - min(DF25_AGG))
ans25 = round(ans25f, 3)

print(ans25f)
print(ans25)

0.2962962962962963
0.296


### Quesetion 26

In [12]:
# 26-1
AID_AGG = CALL.groupby('AID').PASS.mean().reset_index()
AID_LIST = AID_AGG[AID_AGG['PASS']!=0].AID
DF26 = CALL[CALL['AID'].isin(AID_LIST)].copy().reset_index(drop=True)
print(DF26.shape)

(216, 7)


In [13]:
# 26-2
TRAIN26 = DF26[DF26['DAT_ROLE']=='TRAIN'].copy().reset_index(drop=True)
TEST26 = DF26[DF26['DAT_ROLE']=='TEST'].copy().reset_index(drop=True)
print(TRAIN26.shape, TEST26.shape)

(161, 7) (55, 7)


In [14]:
# 26-3
from sklearn.tree import DecisionTreeClassifier
VAR263 = ['CALL_CNT','TM_AVG']
MDL263 = DecisionTreeClassifier(random_state=1234, max_depth=2)
MDL263.fit(TRAIN26[VAR263], TRAIN26['PASS'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1234, splitter='best')

In [15]:
# 26-4
X1 = VAR263[MDL263.tree_.feature[0]]
A6 = MDL263.tree_.threshold[0]
print('X1:',X1)
print('A6:',A6)

X1: TM_AVG
A6: 1199.4400024414062


In [16]:
# 26-5
TEST26['PRED'] = MDL263.predict(TEST26[VAR263])
B6 = TEST26[(TEST26['PASS']==1)&(TEST26['PRED']==1)].shape[0]
print('B6:',B6)

B6: 37


In [17]:
# Answer 26
ans26f = A6 + B6
ans26 = round(ans26f, 1)

print(ans26f)
print(ans26)

1236.4400024414062
1236.4


## Summary

In [18]:
print(ans24)
print(ans25)
print(ans26)

1.429
0.296
1236.4
